In [1]:
### Note :Real represent Propoganda and fake represents non propoganda

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import xticks
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import tensorflow as tf
from sklearn.metrics import f1_score
from wordcloud import WordCloud,STOPWORDS
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding,Activation,Dropout
from keras.layers import Conv1D,MaxPooling1D,GlobalMaxPooling1D,LSTM

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.head()

In [ ]:
dt = df.drop(['Unnamed: 0'],axis=1)

In [ ]:
dt.head()

In [ ]:
from copy import copy
train = dt.copy()

In [ ]:
train.isnull().sum().sort_values(ascending = False)

In [ ]:
# function to draw bar plot
def draw_bar_plot(category,length,xlabel,ylabel,title,sub):
    plt.subplot(2,2,sub)
    plt.bar(category, length)
    plt.legend()
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel(ylabel, fontsize=15)
    plt.title(title, fontsize=15)
    #plt.show()

In [ ]:
# function to draw histogram
def draw_hist(xlabel, ylabel,title,target,sub,color):
    plt.subplot(1,2,sub)
    plt.hist(train[train.target==target]["length"],color = color)
    plt.title(title,fontsize=25)
    plt.xlabel(xlabel,fontsize=15)
    plt.ylabel(ylabel,fontsize=15)
    plt.ylim(0,1200)
    plt.grid()

In [ ]:
# function to draw graphs for stopwords and punctuations
def draw_bar_n_plot(data,title):
# lets visualize the top 10 stop words
    x,y=zip(*data)

    plt.figure(figsize = (25,10))
    plt.subplot(1,2,1)
    plt.bar(x,y)
    plt.title("Top 10 "+ title,fontsize=25)
    plt.xlabel(title,fontsize=15)
    plt.ylabel("Count",fontsize=15)
    plt.grid()

    plt.subplot(1,2,2)
    plt.plot(x,y,'g')
    plt.title("Top 10 "+ title,fontsize=25)
    plt.xlabel(title,fontsize=15)
    plt.ylabel("Count",fontsize=15)
    plt.grid()

In [ ]:
print("No. of propoganda (Target = 1):",len(train[train["target"]==1]))
print("No. of not propoganda (Target = 0):",len(train[train["target"]==0]))

In [ ]:
train["target"].value_counts().plot(kind="bar")

In [ ]:
def length(data):    
    return len(data)

train["length"]= train.data.apply(length)


In [ ]:
plt.figure(figsize = (20,8))
draw_hist("Real Propoganda Data","Length of Data","Length of Real Propoganda Data",1, 1,"darkgreen")
draw_hist("Fake Not propoganda Data","Length of Data","Length of Fake not Propoganda Data",0, 2,"darkred")

In [ ]:
print(train.groupby("target").mean()["length"].sort_values(ascending = False))

In [ ]:
train.drop("length",1,inplace=True)

In [ ]:
train.head()

In [ ]:
train.rename(columns = {'data':'text'}, inplace = True)

In [2]:
stop = list(stopwords.words("english"))

In [3]:
sw = []
for message in train.text:
    for word in message.split():
        if word in stop:
            sw.append(word)


# lets convert the list to a dictinoary which would contain the stop words and their frequency
wordlist = nltk.FreqDist(sw)
# lets save the 10 most frequent stopwords
top10 = wordlist.most_common(10)

NameError: name 'train' is not defined

In [ ]:
draw_bar_n_plot(top10,"Stopwords")

In [ ]:
punctuation = list(string.punctuation)

In [ ]:
pun = []
for message in train.text:
    for word in message.split():
        if word in punctuation:
            pun.append(word)


wordlist = nltk.FreqDist(pun)
# lets save the 10 most frequent stopwords
top10 = wordlist.most_common(10)

In [ ]:
draw_bar_n_plot(top10,"Punctuations")

In [ ]:
stop_real = []
pun_real  = []
for message in train[train.target==1]["text"]:
    for word in message.split():
        if word in stop:
            stop_real.append(word)
        if word in punctuation:
            pun_real.append(word)


stop_real_wordlist = nltk.FreqDist(stop_real)
pun_real_wordlist =  nltk.FreqDist(pun_real)

# lets save the 10 most frequent stopwords
stop_real_top10 = stop_real_wordlist.most_common(10)
pun_real_top10  = pun_real_wordlist.most_common(10)

In [ ]:
stop_fake = []
pun_fake  = []
for message in train[train.target==0]["text"]:
    for word in message.split():
        if word in stop:
            stop_fake.append(word)
        if word in punctuation:
            pun_fake.append(word)


# lets convert the list to a dictinoary which would contain the stop word and its frequency
stop_fake_wordlist = nltk.FreqDist(stop_fake)
pun_fake_wordlist =  nltk.FreqDist(pun_fake)

# lets save the 10 most frequent stopwords
stop_fake_top10 = stop_fake_wordlist.most_common(10)
pun_fake_top10  = pun_fake_wordlist.most_common(10)


In [ ]:
x_stop_real,y_stop_real=zip(*stop_real_top10)
x_pun_real, y_pun_real =zip(*pun_real_top10)

x_stop_fake,y_stop_fake=zip(*stop_fake_top10)
x_pun_fake, y_pun_fake=zip(*pun_fake_top10)


plt.figure(figsize = (30,30))
draw_bar_plot(x_stop_real,y_stop_real,"Stopwords","Count","Top 10 Stopwords - propoganda",1)
draw_bar_plot(x_stop_fake,y_stop_fake,"Stopwords","Count","Top 10 Stopwords - Non Propoganda",2)
draw_bar_plot(x_pun_real,y_pun_real,"Punctuations","Count","Top 10 Punctuations - propoganda",3)
draw_bar_plot(x_pun_fake,y_pun_fake,"Punctuations","Count","Top 10 Punctuations - Non propoganda",4)

In [ ]:
lemma = WordNetLemmatizer()

In [ ]:
sw_pun = stop + punctuation

In [ ]:
# function to preprocess the messages
def preprocess(text):
    text = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", text) # removing urls 
    text = re.sub('[^\w]',' ',text)          
    text = re.sub('[\d]','',text) # this will remove numeric characters
    text = text.lower()
    words = text.split()  
    sentence = ""
    for word in words:     
        if word not in (sw_pun):  # removing stopwords & punctuations                
            word = lemma.lemmatize(word,pos = 'v')  # converting to lemma    
            if len(word) > 3: # we will consider words with length  greater than 3 only
                sentence = sentence + word + ' '             
    return(sentence)

In [ ]:
# apply preprocessing functions on the train and test datasets
train['text'] = train['text'].apply(lambda s : preprocess(s))

In [ ]:
# function to remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
# applying the function on the train and the test datasets
train['text'] = train['text'].apply(lambda s : remove_emoji(s))

In [ ]:
# function to create vocab
from collections import Counter
def create_vocab(df):
    vocab = Counter()
    for i in range(df.shape[0]):
        vocab.update(df.text[i].split())
    return(vocab)

In [ ]:
# concatenate training and testing datasets
#master=pd.concat((train,test)).reset_index(drop=True)

# call vocabulary creation function on master dataset
vocab = create_vocab(train)

# lets check the no. of words in the vocabulary
len(vocab)

In [ ]:
vocab.most_common(50)

In [ ]:
# create the final vocab by considering words with more than one occurence
final_vocab = []
min_occur = 2
for k,v in vocab.items():
    if v >= min_occur:
        final_vocab.append(k)

In [ ]:
len(final_vocab)

In [ ]:
# function to filter the dataset, keep only words which are present in the vocab
def filter(text):
    sentence = ""
    for word in text.split():  
        if word in final_vocab:
            sentence = sentence + word + ' '
    return(sentence)

In [ ]:
train['text'] = train['text'].apply(lambda s : filter(s))

In [ ]:
train.text.head()

In [ ]:
propoganda = train[train.target==1].reset_index()
not_propoganda = train[train.target==0].reset_index()

In [ ]:
# function to create top 100 n-grams
def get_ngrams(data,n):
    all_words = []
    for i in range(len(data)):
        temp = data["text"][i].split()
        for word in temp:
            all_words.append(word)

    tokenized = all_words
    esBigrams = ngrams(tokenized, n)

    esBigram_wordlist = nltk.FreqDist(esBigrams)
    top100 = esBigram_wordlist.most_common(100)
    top100 = dict(top100)
    df_ngrams = pd.DataFrame(sorted(top100.items(), key=lambda x: x[1])[::-1])
    return df_ngrams

In [ ]:
def draw_barplots(real,fake,title):
    plt.figure(figsize = (40,80),dpi=100)

    plt.subplot(1,2,1)
    sns.barplot(y=real[0].values[:100], x=real[1].values[:100], color='green')
    plt.title("Top 100" + title + "in Real Propoganda",fontsize=15)
    
    plt.subplot(1,2,2)
    sns.barplot(y=fake[0].values[:100], x=fake[1].values[:100],color='red')
    plt.title("Top 100" + title + "in Non Propoganda",fontsize=15)

# unigrams

In [ ]:
# lets create top 100 unigrams
real_unigrams = get_ngrams(propoganda,1)
fake_unigrams = get_ngrams(not_propoganda,1)

In [ ]:
draw_barplots(real_unigrams,fake_unigrams," Unigrams ")

# Bigrams

In [ ]:
# lets create top 100 bigrams
real_bigrams = get_ngrams(propoganda,2)
fake_bigrams = get_ngrams(not_propoganda,2)

In [ ]:
draw_barplots(real_bigrams,fake_bigrams," Bigrams ")

# Trigrams

In [ ]:
# lets create top 100 trigrams
real_trigrams = get_ngrams(propoganda,3)
fake_trigrams = get_ngrams(not_propoganda,3)

In [ ]:
draw_barplots(real_trigrams,fake_trigrams," Trigrams ")

In [ ]:
def word_cloud(df):
    comment_words = '' 
    stopwords = set(STOPWORDS) 

    # iterate through the csv file 
    for val in df.text: 

        # typecaste each val to string 
        val = str(val) 

        # split the value 
        tokens = val.split() 
        
        for i in range(len(tokens)): 
            tokens[i] = tokens[i].lower()
        
        comment_words += " ".join(tokens)+" "
        #return comment_words

    wordcloud = WordCloud(width = 800, height = 800, 
            background_color ='white', 
            stopwords = stopwords, 
            min_font_size = 10).generate(comment_words) 
  
    # plot the WordCloud image                        
    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(wordcloud) 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.show() 

In [ ]:
word_cloud(propoganda)

In [ ]:
word_cloud(not_propoganda)

# Model Building & Evaluation


In [ ]:
# function to calculate f1 score for each epoch
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
# Bag of Words model
from keras.preprocessing.text import Tokenizer

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [ ]:
X = train.text
y = train.target

In [ ]:
# Test train split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# create and apply tokenizer on the training dataset
tokenizer = create_tokenizer(X_train)
X_train_set = tokenizer.texts_to_matrix(X_train, mode = 'freq')

In [ ]:
# define the model
def define_model(n_words):
    # define network
    model = Sequential()
    model.add(Dense(128, input_shape=(n_words,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics = [get_f1])
    # summarize defined model
    model.summary()
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [ ]:
# create the model
n_words = X_train_set.shape[1]
model = define_model(n_words)

In [ ]:
model.fit(X_train_set,y_train,epochs=10,verbose=2)

In [ ]:
# prediction on the test dataset
X_test_set = tokenizer.texts_to_matrix(X_test, mode = 'freq')
y_pred = model.predict(X_test_set)

# ANN with KERAS Word Embeddings¶


In [ ]:
# Fitting a tokenizer on text will create a list of unique words with an integer assigned to it
t = Tokenizer()
t.fit_on_texts(X_train.tolist())

In [ ]:
vocab_size = len(t.word_index) + 1


In [ ]:
encoded_docs = t.texts_to_sequences(X_train.tolist())


In [ ]:
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

In [ ]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100,  input_length=100, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[get_f1])
# summarize the model
model.summary()
# fit the model
model.fit(padded_docs, y_train, epochs=50, verbose=0)

In [ ]:
loss, accuracy = model.evaluate(padded_docs, y_train, verbose=0)

In [ ]:
print(accuracy)


# Lstm with word embedding

In [ ]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D


In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(512, embed_dim,input_length = X_train_set.shape[1]))
model.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='RMSprop')
print(model.summary())

In [ ]:
batch_size = 200
hist = model.fit(X_train_set,y_train,epochs = 57, batch_size=batch_size, verbose = 2)

# Bert model

In [ ]:
### Add tokens to the data make it BERT compatible
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
import tensorflow_hub as hub

In [ ]:
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [ ]:
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
train_labels = train.target.values

In [ ]:
model = build_model(bert_layer, max_len=160)
model.summary()

In [ ]:
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    batch_size=16
)